In [ ]:
using TSML
using DataFrames
using Dates
using CSV
using Random

In [2]:
fname ="testdata.csv"
dat = CSV.read(fname)
rename!(dat,names(dat)[1]=>:Date,names(dat)[2]=>:Value)
dat.Date = DateTime.(dat.Date,"d/m/y H:M")
orig = deepcopy(dat)
filter1 = DateValgator()
filter2 = DateValNNer(Dict(:nnsize=>1))

DateValNNer(nothing, Dict{Symbol,Any}(:strict => true,:dateinterval => 1 hour,:aggregator => :median,:nnsize => 1,:missdirection => :symmetric))

In [3]:
fit!(filter1,dat,[])
res1=transform!(filter1,dat)
(first(res1,5),size(res1,1))

(5×2 DataFrame
│ Row │ Date                │ Value    │
│     │ DateTime            │ Float64⍰ │
├─────┼─────────────────────┼──────────┤
│ 1   │ 2014-01-01T00:00:00 │ 10.0     │
│ 2   │ 2014-01-01T01:00:00 │ 9.9      │
│ 3   │ 2014-01-01T02:00:00 │ 10.0     │
│ 4   │ 2014-01-01T03:00:00 │ 10.0     │
│ 5   │ 2014-01-01T04:00:00 │ 10.0     │, 8761)

In [4]:
fit!(filter2,res1,[])
res2=transform!(filter2,res1)
(first(res2,5),size(res2,1))

(5×2 DataFrame
│ Row │ Date                │ Value    │
│     │ DateTime            │ Float64⍰ │
├─────┼─────────────────────┼──────────┤
│ 1   │ 2014-01-01T00:00:00 │ 10.0     │
│ 2   │ 2014-01-01T01:00:00 │ 9.9      │
│ 3   │ 2014-01-01T02:00:00 │ 10.0     │
│ 4   │ 2014-01-01T03:00:00 │ 10.0     │
│ 5   │ 2014-01-01T04:00:00 │ 10.0     │, 8761)

In [5]:
mypipeline = Pipeline(Dict(
        :transformers => [filter1,filter2]
    )
)

Pipeline(nothing, Dict{Symbol,Union{Nothing, Array{Transformer,1}}}(:transformers => Transformer[DateValgator(Dict{Symbol,Any}(:dateinterval => 1 hour,:aggregator => :median), Dict{Symbol,Any}(:dateinterval => 1 hour,:aggregator => :median)), DateValNNer(Dict{Symbol,Any}(:loopcount => 1,:strict => true,:dateinterval => 1 hour,:missingcount => 4931,:aggregator => :median,:nnsize => 1,:missdirection => :symmetric), Dict{Symbol,Any}(:loopcount => 1,:strict => true,:dateinterval => 1 hour,:missingcount => 4931,:aggregator => :median,:nnsize => 1,:missdirection => :symmetric))],:transformer_args => nothing))

In [6]:
fit!(mypipeline,dat,[])
resp = transform!(mypipeline,dat)
first(resp,5)

,Date,Value
,DateTime,Float64⍰
1,2014-01-01T00:00:00,10.0
2,2014-01-01T01:00:00,9.9
3,2014-01-01T02:00:00,10.0
4,2014-01-01T03:00:00,10.0
5,2014-01-01T04:00:00,10.0


In [7]:
using TSML.TSMLTypes
import TSML.TSMLTypes.fit!
import TSML.TSMLTypes.transform!

mutable struct MyCSVDateValReader <: Transformer
    model
    args
    function MyCSVDateValReader(args=Dict())
        default_args = Dict(
            :filename => "",
            :dateformat => ""
        )
        new(nothing,mergedict(default_args,args))
    end
end
function fit!(csvrdr::MyCSVDateValReader,x::T=[],y::Vector=[]) where {T<:Union{DataFrame,Vector,Matrix}}
    fname = csvrdr.args[:filename]
    fmt = csvrdr.args[:dateformat]
    (fname != "" && fmt != "") || error("missing filename or date format")
    model = csvrdr.args
end

function transform!(csvrdr::MyCSVDateValReader,x::T=[]) where {T<:Union{DataFrame,Vector,Matrix}}
    fname = csvrdr.args[:filename]
    fmt = csvrdr.args[:dateformat]
    df = CSV.read(fname)
    ncol(df) == 2 || error("dataframe should have only two columns: Date,Value")
    rename!(df,names(df)[1]=>:Date,names(df)[2]=>:Value)
    df.Date = DateTime.(df.Date,fmt)
    df
end

transform! (generic function with 42 methods)

In [8]:
csvreader = MyCSVDateValReader(Dict(:filename=>"testdata.csv",:dateformat=>"d/m/y H:M"))
fit!(csvreader)
res=transform!(csvreader)
first(res,5)

,Date,Value
,DateTime,Float64
1,2014-01-01T00:06:00,10.0
2,2014-01-01T00:18:00,10.0
3,2014-01-01T00:29:00,10.0
4,2014-01-01T00:40:00,9.9
5,2014-01-01T00:51:00,9.9


In [9]:
mypipeline = Pipeline(Dict(
        :transformers => [csvreader,filter1,filter2]
    )
)
fit!(mypipeline)
res=transform!(mypipeline)
first(res,5)

,Date,Value
,DateTime,Float64⍰
1,2014-01-01T00:00:00,10.0
2,2014-01-01T01:00:00,9.9
3,2014-01-01T02:00:00,10.0
4,2014-01-01T03:00:00,10.0
5,2014-01-01T04:00:00,10.0


In [10]:
filter2.args[:missingcount]

4931

In [11]:
mydate=DateTime(2014,1,1):Dates.Minute(15):DateTime(2014,1,3) # 15 minutes interval
values = Array{Union{Float64,Missing}}(sin.(1:0.1:length(mydate)) .+ cos.(1:0.1:length(mydate)))[1:length(mydate)];
x = DataFrame(Date=mydate,Value=values); xx = deepcopy(x);first(x,10)
nmissing=floor(nrow(x) * 0.80) |> Integer
ndxmissing=Random.shuffle(1:nrow(x))[1:nmissing]
x.Value[ndxmissing] .= missing; first(x,15)
dvtr = TSML.DateValgator()
TSML.fit!(dvtr,x,[]);TSML.fit!(dvtr,xx,[])
inputx = TSML.transform!(dvtr,x); inputxx = TSML.transform!(dvtr,xx);
dvnnr = TSML.DateValNNer(Dict(:dateinterval=>Dates.Hour(1),:nnsize=>1))
TSML.fit!(dvnnr,inputx,[])
pred_y=TSML.transform!(dvnnr,inputx);pred_yy=TSML.transform!(dvnnr,inputxx);

In [12]:
sqrt(sum(pred_y.Value-pred_yy.Value).^2)

0.522726120677754

In [13]:
rename!(pred_y,:Value=>:MissingVals);

In [14]:
jx = join(pred_y,pred_yy,on=:Date)
jx.error = (jx.MissingVals .- jx.Value).^2;jx

,Date,MissingVals,Value,error
,DateTime,Float64⍰,Float64⍰,Float64
1,2014-01-01T00:00:00,0.746646,1.36329,0.380248
2,2014-01-01T01:00:00,0.746646,1.19324,0.199444
3,2014-01-01T02:00:00,0.746646,0.804733,0.00337414
4,2014-01-01T03:00:00,-0.105454,0.289179,0.155736
5,2014-01-01T04:00:00,-0.957554,-0.272029,0.469945
6,2014-01-01T05:00:00,-0.957554,-0.790291,0.0279772
7,2014-01-01T06:00:00,-1.18563,-1.18378,3.40069e-6
8,2014-01-01T07:00:00,-1.4137,-1.39038,0.000543709
9,2014-01-01T08:00:00,-1.38614,-1.37747,7.52128e-5
